In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['RK']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('rhodopsin kinase', 31),
 ('raspberry ketone', 18),
 ('radial keratotomy', 9),
 ('remnant kidney', 9),
 ('rabbit kidney', 8),
 ('reactivating kinase', 6),
 ('ribokinase', 5),
 ('radial kurtosis', 4),
 ('rye bread with kernels', 4),
 ('rapid kindling', 3),
 ('rye kernels', 3)]

In [14]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

In [16]:
result

[{'rabbit kidney': 'ungrounded',
  'radial keratotomy': 'MESH:D007646',
  'radial kurtosis': 'ungrounded',
  'rapid kindling': 'ungrounded',
  'raspberry ketone': 'CHEBI:CHEBI:68656',
  'reactivating kinase': 'FPLX:p38',
  'remnant kidney': 'ungrounded',
  'rhodopsin kinase': 'MESH:D051606',
  'ribokinase': 'HGNC:30325',
  'rye bread with kernels': 'ungrounded',
  'rye kernels': 'ungrounded'},
 {'MESH:D007646': 'Keratotomy, Radial',
  'CHEBI:CHEBI:68656': 'raspberry ketone',
  'FPLX:p38': 'p38',
  'MESH:D051606': 'G-Protein-Coupled Receptor Kinase 1',
  'HGNC:30325': 'RBKS'},
 ['CHEBI:CHEBI:68656',
  'FPLX:p38',
  'HGNC:30325',
  'MESH:D007646',
  'MESH:D051606']]

In [18]:
grounding_map, names, pos_labels = [{'rabbit kidney': 'ungrounded',
  'radial keratotomy': 'MESH:D007646',
  'radial kurtosis': 'ungrounded',
  'rapid kindling': 'ungrounded',
  'raspberry ketone': 'CHEBI:CHEBI:68656',
  'reactivating kinase': 'FPLX:p38',
  'remnant kidney': 'ungrounded',
  'rhodopsin kinase': 'MESH:D051606',
  'ribokinase': 'HGNC:30325',
  'rye bread with kernels': 'ungrounded',
  'rye kernels': 'ungrounded'},
 {'MESH:D007646': 'Keratotomy, Radial',
  'CHEBI:CHEBI:68656': 'raspberry ketone',
  'FPLX:p38': 'p38',
  'MESH:D051606': 'G-Protein-Coupled Receptor Kinase 1',
  'HGNC:30325': 'RBKS'},
 ['CHEBI:CHEBI:68656',
  'FPLX:p38',
  'HGNC:30325',
  'MESH:D007646',
  'MESH:D051606']]

In [21]:
excluded_longforms = []

In [22]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [23]:
additional_entities = {}

In [24]:
unambiguous_agent_texts = {}

In [25]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [12]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [13]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [14]:
intersection1

[('HGNC:1848', 'HGNC:1848', 86)]

In [15]:
intersection2

[('MESH:D018893', 'HGNC:1848', 0),
 ('HGNC:1848', 'HGNC:1848', 1),
 ('ungrounded', 'HGNC:1848', 0),
 ('CHEBI:CHEBI:64198', 'HGNC:1848', 0),
 ('MESH:D019164', 'HGNC:1848', 0),
 ('MESH:C059416', 'HGNC:1848', 0),
 ('CHEBI:CHEBI:15710', 'HGNC:1848', 0)]

In [1]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

NameError: name 'unambiguous_agent_texts' is not defined

In [18]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [26]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-29 02:39:06] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-29 02:39:09] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.916969696969697 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [27]:
classifier.stats

{'label_distribution': {'MESH:D051606': 29,
  'ungrounded': 18,
  'FPLX:p38': 4,
  'MESH:D007646': 7,
  'CHEBI:CHEBI:68656': 11,
  'HGNC:30325': 3},
 'f1': {'mean': 0.91697, 'std': 0.053375},
 'precision': {'mean': 0.934545, 'std': 0.055566},
 'recall': {'mean': 0.907273, 'std': 0.057611},
 'ungrounded': {'f1': {'mean': 0.84127, 'std': 0.143385},
  'pr': {'mean': 0.933333, 'std': 0.133333},
  'rc': {'mean': 0.77, 'std': 0.16}},
 'CHEBI:CHEBI:68656': {'f1': {'mean': 0.813333, 'std': 0.243676},
  'pr': {'mean': 0.766667, 'std': 0.290593},
  'rc': {'mean': 0.9, 'std': 0.2}},
 'MESH:D007646': {'f1': {'mean': 1.0, 'std': 0.0},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'FPLX:p38': {'f1': {'mean': 0.8, 'std': 0.4},
  'pr': {'mean': 0.8, 'std': 0.4},
  'rc': {'mean': 0.8, 'std': 0.4}},
 'HGNC:30325': {'f1': {'mean': 0.2, 'std': 0.4},
  'pr': {'mean': 0.2, 'std': 0.4},
  'rc': {'mean': 0.2, 'std': 0.4}},
 'MESH:D051606': {'f1': {'mean': 1.0, 'std': 0.0},
  'pr': {'

In [28]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [29]:
disamb.dump(model_name, results_path)

In [30]:
print(disamb.info())

Disambiguation model for RK

Produces the disambiguations:
	G-Protein-Coupled Receptor Kinase 1*	MESH:D051606
	Keratotomy, Radial*	MESH:D007646
	RBKS*	HGNC:30325
	p38*	FPLX:p38
	raspberry ketone*	CHEBI:CHEBI:68656

Class level metrics:
--------------------
Grounding                          	Count	F1     
G-Protein-Coupled Receptor Kinase 1*	29	    1.0
                         Ungrounded	18	0.84127
                   raspberry ketone*	11	0.81333
                 Keratotomy, Radial*	 7	    1.0
                                p38*	 4	    0.8
                               RBKS*	 3	    0.2

Weighted Metrics:
-----------------
	F1 score:	0.91697
	Precision:	0.93454
	Recall:		0.90727

* Positive labels
See Docstring for explanation



In [31]:
model_to_s3(disamb)